#### ▩ 1. seq2seq 의 문제점은 무엇인가요 ?
    고정길이 벡터로 변환해야만 한다
![fig](http://cfile265.uf.daum.net/image/996DF2395F576B82063808)

#### ▩ 2. 그러면 어떻게 개선을 해야하나요 ?
    seq2seq를 이루고 있는 Encoder와 Decoder를 개선
    Encoder 출력의 길이를 입력 문장의 길이에 따라 바꿔준다
        Encoder는 아무리 긴 문장이라도 하나의 고정길이 벡터로 변환. 개선된 Encoder는 입력된 단어와 같은 수의 벡터를 얻을 수 있다.
![fig](http://cfile289.uf.daum.net/image/990CAC395F576D2E05FE3C)

#### ▩ 3. 위의 그림 8-2 에서 주목할 내용이 무엇입니까 ?
    LSTM 계층의 은닉 상태의 '내용'
        은닉 상태의 내용은 각 시각의 직전에 입력된 단어에 대한 정보가 많이 포함되어져 있다.
        바로 직전에 '고양이'라는 단어를 입력했다면 이때 출력되는 은닉 상태 벡터는 '고양이' 성분이 많이 들어간 벡터라고 생각할 수 있다.
![fig](http://cfile277.uf.daum.net/image/99C299345F5771D10E0EAF)

#### ▩ 4. Decoder 는 어떻게 개선해야하나요? 
    Encoder는 각 단어에 대응하는 LSTM 계층의 은닉상태 벡터를 hs로 모아 출력
    hs가 Decoder에 전달되어 시계열 변환이 이루어진다.
    Encoder의 LSTM 계층의 '마지막' 은닉상태를 Decoder의 LSTM 계층의 '첫' 은닉상태로 설정
![fig](http://cfile292.uf.daum.net/image/99A4EC4F5F5789A9055A94)

    인코더가 단어에 해당하는 벡터들의 집합을 hs 로 리턴했으면 이 집합을 다 활용해야하는데 그런데 아래의 그림 처럼 hs 에서 마지막 줄만 빼내어 Decoder 에게 전달 합니다. 바로 이부분을 개선해야합니다. 다 활용되게 해야합니다. 
![fig](http://cfile254.uf.daum.net/image/99810E4F5F578A7E0EA7B1)

#### ▩ 5. 그러면 어떻게 해야 Encoder 의 hs 를 다 활용하게 할 수 있을까요?
    나 = I, 고양이 = cat 처럼 입력과 출력의 여러 단어 중 어떤 단어끼리 서로 관련되어 있는가 라는 대응관계를 seq2seq에게 학습시키면 된다

#### ▩ 6. 이런 대응관계를 나타내는 정보를 무엇이라고 합니까?
    얼라인먼트(Alignment)
        지금까지는 사람이 수작업으로 했는데 이제는 기계를 통한 자동화 되어지고 있다 -> 어텐션

#### ▩ 7. 앞으로 우리의 목표는 무엇입니까?
    도착어 단어와 대응관계에 있는 출발어 단어의 정보를 골라내는 것
        예: cat(도착어) = 고양이(출발어)
            이 정보를 이용하여 번역을 수행

#### ▩ 8. 이렇게 필요한 정보에만 주목하여 그 정보로 부터 시계열 변환을 수행하는 것을 무엇이라고 합니까?
    Attention

#### ▩ 9. 아래의 그림 8-6의 신경망으로 하고 싶은일은 단어들의 얼라인먼트 추출합니다. 각 시각에서 Decoder 에 입력된 단어와 대응관계인 단어의 백터를 hs 에서 골라내겠다는 뜻입니다.  이러한 선택작업이 아래의 그림에서 어느 부분입니까 ?
    어떤 계산 부분. 
    Decoder가 'I'를 출력할 때 hs에서 '나'에 대해 대응하는 벡터를 선택하게 하면 되는데 이 선택 작업을 '어떤 계산'으로 해낸다
![fig](http://cfile283.uf.daum.net/image/99E28A4B5F578EBC13A982)

#### ▩ 10. decoder 가 I 를 출력할 때 hs 에서 '나' 에 대응하는 벡터를 선택하게 하려면 구체적으로 어떻게 구현해야 할까요 ?
    각 단어에 대해서 그것이 얼마나 중요한지를 나타내는 '가중치'를 이용
        '나'에 해당되는 가중치는 0.8
![fig](http://cfile270.uf.daum.net/image/995A84465F579209149F19)

#### ▩ 11. 그림 8-6의 어떤 계산은 무엇입니까 ?
    단어의 벡터 hs와 가중치 a와의 가중합을 구하여 원하는 벡터 c를 얻는 것
        c : 맥락 벡터. '나'에 대응하는 가중치가 0.8이므로 이 맥락 벡터에는 '나' 벡터 성분이 많이 포함되어 있다.
![fig](http://cfile297.uf.daum.net/image/991517475F5793610DFA4B)

#### ▩ 12. 지금까지의 애기를 코드로 구현해 봅니다. 

In [ ]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5, 1).repeat(4, axis=1)
print(ar.shape)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=0)
print(c)
print(c.shape)

### (점심시간 문제) 하나의 문장에 대해서만 맥락벡터를 구한 것이고 점심시간 문제는 배치단위로 구현하는 것

In [ ]:
# 미치배치 10개로 hs를 생성
hs = np.random.randn(10,5,4)
a = np.random.randn(10,5)
ar = a.reshape(10,5,4)

#### ▩ 14. 그럼 가중치 a 는 어떻게 구해야하는가요?
    데이터로부터 자동으로 학습되어져 구해질 수 있도록 해야한다

#### ▩ 15. 그럼 어떻게 해야 가중치 a 가 자동으로 구해질 수 있는것인가요 ?
    아래의 그림과 같이 hs의 5개의 벡터에는 각각 입력된 단어에 대한 정보가 많이 포함되어져 있다.
![fig](http://cfile299.uf.daum.net/image/99F39D3F5F594A23081566)

    위의 hs를 decoder 계층에서 전부 사용하면서 decoder에 입력된 단어와 대응관계인 단어의 벡터를 hs에서 골라내려는 alignment를 추출
    아래와 같이 계산이 이루어지는데
![fig](http://cfile268.uf.daum.net/image/9915B7405F5949ED0A47C2)
    
    위 계산은 다음 그림과 같이 각 단어의 중요도(기여도)를 나타내는 '가중치 a'를 별도로 계산하도록해서 hs와 a의 가중합으로 맥락벡터를 얻어낸다
    맥락벡터에는 각 단어에 대한 성분이 많이 포함되어져 있다. 
    따라서 맥락벡터(c)를 통해서 '나'가 선택되는 맥락벡터(c)가 출력층에 들어오면 'I'가 선택될 확률이 높아지도록 출력층의 가중치인 W가 학습되어지도록 한다.
![fig](http://cfile251.uf.daum.net/image/9901BF395F594BF307FCEF)

    가중치 a가 자동으로 계산되어지게 해야하는데 이 a를 자동으로 알아내야하는 가중치로 본다면 이 가중치가 좋은 가중치인지 아닌지를 먼저 판단
    좋은 가중치가 되도록 자동으로 갱신되어져야 한다. 좋은 가중치인지 판단하는 것은 h가 hs의 각 단어벡터와 얼마나 '비슷한가' 수치로 나타내는데
    이 때 벡터의 내적을 이용
![fig2](http://cfile292.uf.daum.net/image/99CD44385F5949400993A4)

    학습을 시키게 되면 점점 softmax의 결과가 해당 단어의 확률값이 점점 1에 가까워지고 나머지는 0에 가까워지게끔 학습이 된다.
![fig3](http://cfile277.uf.daum.net/image/99155A405F59495D0A7620)
![fig4](http://cfile250.uf.daum.net/image/99B8113D5F59513A240E40)

    

#### ▩ 16. 지금까지의 순서를 종합한 아래의 그림으로 Attention 을 설명하세요 
![fig](http://cfile277.uf.daum.net/image/993508335F59520D0ACC68)

    Encoder에서 출력된 단어들의 정보를 담은 hs가 decoder의 특정 시계열 시점에서 LSTM층에서 출력된 h와의 유사도를 구해서
    그 유사도를 softmax 함수를 통과시켜 가중치(기여도) a를 구하고 a는 hs과 곱해져서 맥락벡터인 c를 산출
    
![fig](http://cfile248.uf.daum.net/image/992A54355F5952280BC264)

    

#### ▩ 17. 결국 아래의 그림의 Attention 층의 역할이 무엇인가요 ?
    Encoder가 산출한 hs를 Decoder의 모든 시계열 t 시점에서 다 활용할 수 있도록 hs에서 필요한 정보(특정 벡터)를 골라내기 위해 집중시키는 역할

#### Attention Chatbot

In [ ]:
# !pip install customized_konlpy
# from google.colab import drive
# drive.mount('/content/gdrive')

from ckonlpy.tag import Twitter
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from nltk import FreqDist
from functools import reduce
import os
import re
import matplotlib.pyplot as plt

# 훈련 데이터 : https://bit.ly/31SqtHy
# 테스트 데이터 : https://bit.ly/3f7rH5g

TRAIN_FILE = os.path.join("D:/Desktop/Itwill ws/rnn/qa1_single-supporting-fact_train_kor.txt")
TEST_FILE = os.path.join("D:/Desktop/Itwill ws/rnn/qa1_single-supporting-fact_test_kor.txt")

# conda install nltk

i = 0
lines = open(TRAIN_FILE, "rb")
for line in lines:
    line = line.decode("utf-8").strip()
    i = i + 1
    print(line)
    if i == 20:
        break


def read_data(dir):
    stories, questions, answers = [], [], []  # 각각 스토리, 질문, 답변을 저장할 예정
    story_temp = []  # 현재 시점의 스토리 임시 저장
    lines = open(dir, "rb")

    for line in lines:
        line = line.decode("utf-8")  # b' 제거
        line = line.strip()  # '\n' 제거
        idx, text = line.split(" ", 1)  # 맨 앞에 있는 id number 분리
        # 여기까지는 모든 줄에 적용되는 전처리

        if int(idx) == 1:
            story_temp = []

        if "\t" in text:  # 현재 읽는 줄이 질문 (tab) 답변 (tab)인 경우
            question, answer, _ = text.split("\t")  # 질문과 답변을 각각 저장
            stories.append([x for x in story_temp if x])  # 지금까지의 누적 스토리를 스토리에 저장
            questions.append(question)
            answers.append(answer)

        else:  # 현재 읽는 줄이 스토리인 경우
            story_temp.append(text)  # 임시 저장

    lines.close()
    return stories, questions, answers


# %%

train_data = read_data(TRAIN_FILE)
test_data = read_data(TEST_FILE)

# %%
train_stories, train_questions, train_answers = read_data(TRAIN_FILE)
test_stories, test_questions, test_answers = read_data(TEST_FILE)


print('훈련용 스토리의 개수 :', len(train_stories))
print('훈련용 질문의 개수 :', len(train_questions))
print('훈련용 답변의 개수 :', len(train_answers))
print('테스트용 스토리의 개수 :', len(test_stories))
print('테스트용 질문의 개수 :', len(test_questions))
print('테스트용 답변의 개수 :', len(test_answers))

print(train_stories[3572])

print(train_questions[3572])

print(train_answers[3572])


# %%

# def tokenize(sent):
#     print(re.split('(\W+)?', sent))
#     return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def tokenize(sent):
    return [x.strip() for x in re.split(r'(\W+)?', sent) if x and x.strip()]


# %%

def preprocess_data(train_data, test_data):
    counter = FreqDist()

    # 두 문장의 story를 하나의 문장으로 통합하는 함수
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    # 각 샘플의 길이를 저장하는 리스트
    story_len = []
    question_len = []

    for stories, questions, answers in [train_data, test_data]:
        for story in stories:
            stories = tokenize(flatten(story))  # 스토리의 문장들을 펼친 후 토큰화
            story_len.append(len(stories))  # 각 story의 길이 저장
            for word in stories:  # 단어 집합에 단어 추가
                counter[word] += 1
        for question in questions:
            question = tokenize(question)
            question_len.append(len(question))
            for word in question:
                counter[word] += 1
        for answer in answers:
            answer = tokenize(answer)
            for word in answer:
                counter[word] += 1

    # 단어 집합 생성
    word2idx = {word: (idx + 1) for idx, (word, _) in enumerate(counter.most_common())}
    idx2word = {idx: word for word, idx in word2idx.items()}

    # 가장 긴 샘플의 길이
    story_max_len = np.max(story_len)
    question_max_len = np.max(question_len)

    return word2idx, idx2word, story_max_len, question_max_len


# %%

word2idx, idx2word, story_max_len, question_max_len = preprocess_data(train_data, test_data)
print(word2idx)

twitter = Twitter()

twitter.add_dictionary('은경이', 'Noun')
twitter.add_dictionary('경임이', 'Noun')
twitter.add_dictionary('수종이', 'Noun')


# print(twitter.morphs('은경이는 화장실로 이동했습니다.'))
# print(twitter.morphs('경임이는 정원으로 가버렸습니다.'))
# print(twitter.morphs('수종이는 복도로 뛰어갔습니다.'))
# print(twitter.morphs('필웅이는 부엌으로 복귀했습니다.'))
# print(twitter.morphs('수종이는 사무실로 갔습니다.'))
# print(twitter.morphs('은경이는 침실로 갔습니다.'))


def tokenize(sent):
    return twitter.morphs(sent)


word2idx, idx2word, story_max_len, question_max_len = preprocess_data(train_data, test_data)
# print(word2idx)

vocab_size = len(word2idx) + 1


# print(vocab_size)

# print('스토리의 최대 길이 :', story_max_len)
# print('질문의 최대 길이 :', question_max_len)


def vectorize(data, word2idx, story_maxlen, question_maxlen):
    Xs, Xq, Y = [], [], []
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    stories, questions, answers = data
    for story, question, answer in zip(stories, questions, answers):
        xs = [word2idx[w] for w in tokenize(flatten(story))]
        xq = [word2idx[w] for w in tokenize(question)]
        Xs.append(xs)
        Xq.append(xq)
        Y.append(word2idx[answer])

        # 스토리와 질문은 각각의 최대 길이로 패딩
        # 정답은 원-핫 인코딩
    return pad_sequences(Xs, maxlen=story_maxlen), \
           pad_sequences(Xq, maxlen=question_maxlen), \
           to_categorical(Y, num_classes=len(word2idx) + 1)


Xstrain, Xqtrain, Ytrain = vectorize(train_data, word2idx, story_max_len, question_max_len)
Xstest, Xqtest, Ytest = vectorize(test_data, word2idx, story_max_len, question_max_len)

# print(Xstrain.shape, Xqtrain.shape, Ytrain.shape, Xstest.shape, Xqtest.shape, Ytest.shape)

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Permute, dot, add, concatenate
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Activation

# 에포크 횟수
train_epochs = 120
# 배치 크기
batch_size = 32
# 임베딩 크기
embed_size = 50
# LSTM의 크기
lstm_size = 64
# 과적합 방지 기법인 드롭아웃 적용 비율
dropout_rate = 0.30

# 플레이스 홀더. 입력을 담는 변수
input_sequence = Input((story_max_len,))
question = Input((question_max_len,))

# print('Stories :', input_sequence)
# print('Question:', question)

# 스토리를 위한 첫번째 임베딩. 그림에서의 Embedding A
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=embed_size))
input_encoder_m.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, embedding_dim) / 샘플의 수, 문장의 최대 길이, 임베딩 벡터의 차원

# 스토리를 위한 두번째 임베딩. 그림에서의 Embedding C
# 임베딩 벡터의 차원을 question_max_len(질문의 최대 길이)로 한다.
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=question_max_len))
input_encoder_c.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이(임베딩 벡터의 차원)

# 질문을 위한 임베딩. 그림에서의 Embedding B
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=embed_size,
                               input_length=question_max_len))
question_encoder.add(Dropout(dropout_rate))
# 결과 : (samples, question_max_len, embedding_dim) / 샘플의 수, 질문의 최대 길이, 임베딩 벡터의 차원

# 실질적인 임베딩 과정
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

print('Input encoded m', input_encoded_m)
print('Input encoded c', input_encoded_c)
print('Question encoded', question_encoded)

# 스토리 단어들과 질문 단어들 간의 유사도를 구하는 과정
# 유사도는 내적을 사용한다.
match = dot([input_encoded_m, question_encoded], axes=-1, normalize=False)
match = Activation('softmax')(match)
print('Match shape', match)
# 결과 : (samples, story_maxlen, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_max_len, question_max_len)
response = Permute((2, 1))(response)  # (samples, question_max_len, story_max_len)
print('Response shape', response)

# concatenate the response vector with the question vector sequence
answer = concatenate([response, question_encoded])
print('Answer shape', answer)

answer = LSTM(lstm_size)(answer)  # Generate tensors of shape 32
answer = Dropout(dropout_rate)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])

print(model.summary())

# start training the model
history = model.fit([Xstrain, Xqtrain],
                    Ytrain, batch_size, train_epochs,
                    validation_data=([Xstest, Xqtest], Ytest))

# save model
model.save('model.h5')

# plot accuracy and loss plot
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

# labels
ytest = np.argmax(Ytest, axis=1)

# get predictions
Ytest_ = model.predict([Xstest, Xqtest])
ytest_ = np.argmax(Ytest_, axis=1)

NUM_DISPLAY = 30

print("{:18}|{:5}|{}".format("질문", "실제값", "예측값"))
print(39 * "-")

for i in range(NUM_DISPLAY):
    question = " ".join([idx2word[x] for x in Xqtest[i].tolist()])
    label = idx2word[ytest[i]]
    prediction = idx2word[ytest_[i]]
    print("{:20}: {:7} {}".format(question, label, prediction))
